#### What Require Inputs in this Notebook
1. [Fill in File Input](#Load-in-Data)
2. [Right or Left Hand](#Right-or-Left-Hand)
3. [Output ShotCSV](#Output-ShotCSV)
4. [Change to atNet (by Player) from tagger later](#At-Net)
5. [EDA and get Stats Here](#EDA)
6. [Output PointCSV for Visuals](#PRINT-POINTCSV-VISUALS)
7. [Output PointCSV for Upload](#Print-POINTCSV-UPLOAD)

# Table of Contents
1. [Load in Data](#Load-in-Data)
- Error Checking
- Add Columns
- [Output ShotCSV](#Output-ShotCSV)
2. [Create Point DF](#Create-Point-DF)
- Add Columns
- [Output PointCSV for Visuals](#PRINT-POINTCSV-VISUALS)
- [Output PointCSV for Upload](#Print-POINTCSV-UPLOAD)    
        - cut out points with no timestamp position    
        - atNetPlayer1 has values of the player name for display purposes instead of boolean values

# [Click for Summary Stats and EDA HERE](#EDA)

### List of Changes

For Shot CSV
- Adds Point Number (Comment out if already there)
- Changes ShotType to ShotDirection and ShotFhBh (old tagger)
- Forward Fills these columns'gameScore','setScore', 'tiebreakScore', 'serverName', 'player1Name', 'player2Name'

Columns Added:
  - 'isDoubleFault',
  - 'pointWonBy',
  - 'lastShotError',
  - 'serveResult',
  - 'isInsideIn',
  - 'returnerName',
  - 'serveInPlacement',
  - 'player1Hand',
  - 'player2Hand',
  - 'shotHitBy',
  - 'isInsideOut'
  

For Point (Visuals)
- point related info, and player1 focused stats


For Point (Upload)
- removes columns with no start time
- changes "1" value of atNetPlayer1 to player1Name

### Warning: Changes to be made soon
- atNetPlayer1, atNetPlayer2 are being aggregated by if hitting a volley. Will be replaced by tagger values for those
- pointWonBy is being calculated, replaced by tagger soon
- player1Hand and player2Hand are being manually inputted

### Future Changes to Notebook
- More stats on approach shots
- Pressure Point stats, like what did players did on 30-30 points onwards
- Inside Out/Inside In Stats once we get matches with accurate side info
- handling of rows where there is error on video side and the point is cut short

Notes:
- Keep empty values None Type, they will be converted to empty string "" right before outputting to Point CSV Only for Upload
- Having 1 does not mean other value is boolean 0, it is None

# Notebook Start

### Load in Data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Put your file name here

your_file_name = "yourFileNameHere"
shot_data = pd.read_csv(your_file_name)



In [ ]:
# Check existing columns

shot_data.columns

<!-- ### Error Check: 
#### If data includes firstServeZone but no "1" for firstServeIn, same for secondServeZone/secondServeIn -->

### Error Check: 
#### If returnData exists, separate values of "Forehand/Backhand Crosscourt/Down the Line" into shotFhBh and shotDirection

In [ ]:
# Check if 'returnData' column exists
if 'returnData' in shot_data.columns:
    # Replace NaN values in returnData column with an empty string
    shot_data['returnData'].fillna('', inplace=True)

    # Check if "backhand" or "forehand" is in returnData and update shotFhBh accordingly
    shot_data.loc[shot_data['returnData'].str.contains('backhand', case=False), 'shotFhBh'] = 'Backhand'
    shot_data.loc[shot_data['returnData'].str.contains('forehand', case=False), 'shotFhBh'] = 'Forehand'

    # Check if "Crosscourt" or "Down the Line" is in returnData and update shotDirection accordingly
    shot_data.loc[shot_data['returnData'].str.contains('Crosscourt', case=False), 'shotDirection'] = 'Crosscourt'
    shot_data.loc[shot_data['returnData'].str.contains('Down the Line', case=False), 'shotDirection'] = 'Down the Line'

    # Display the modified shot_data
    print(shot_data[['returnData', 'shotDirection', 'shotFhBh']].head(10))
else:
    print("Column 'returnData' does not exist.")


### Error Check: 
#### Change ShotType to shotFhBh if old tagger

In [ ]:
if 'shotType' in shot_data.columns:
    # Rename the column
    shot_data.rename(columns={'shotType': 'shotFhBh'}, inplace=True)
    print("Column renamed successfully.")
else:
    print("Check Passed: Column 'shotType' does not exist.")

### Error Check: 
#### From Old Tagger, checks for shotFhBh doesn't include direction and fh/bh

In [ ]:
shot_data.loc[shot_data['shotFhBh'] == 'Forehand Crosscourt', 'shotDirection'] = 'Crosscourt'
shot_data.loc[shot_data['shotFhBh'] == 'Forehand Crosscourt', 'shotFhBh'] = 'Forehand'

shot_data.loc[shot_data['shotFhBh'] == 'Backhand Crosscourt', 'shotDirection'] = 'Crosscourt'
shot_data.loc[shot_data['shotFhBh'] == 'Backhand Crosscourt', 'shotFhBh'] = 'Backhand'

shot_data.loc[shot_data['shotFhBh'] == 'Forehand Down the Line', 'shotDirection'] = 'Down the Line'
shot_data.loc[shot_data['shotFhBh'] == 'Forehand Down the Line', 'shotFhBh'] = 'Forehand'

shot_data.loc[shot_data['shotFhBh'] == 'Backhand Down the Line', 'shotDirection'] = 'Down the Line'
shot_data.loc[shot_data['shotFhBh'] == 'Backhand Down the Line', 'shotFhBh'] = 'Backhand'

print(shot_data[['shotDirection', 'shotFhBh']].head(10))

### Error Check: 
#### Checks Player1Name and Player2Name have Values

In [ ]:
fillInPlayer1Name = "Choose_Player1Name_Here"
fillInPlayer2Name = "Choose_Player2Name_Here"

if shot_data.at[0, 'player1Name'] == None:
    print("player1Name was blank. Filling with 'fillInplayer1Name'")
    shot_data.at[0, 'player1Name'] = fillInPlayer1Name
else:
    print("Check Passed: player1Name is not blank. Current value:", shot_data.at[0, 'player1Name'])

# Check if player2Name is blank, if so, fill it with 'fillInplayer2Name'
if shot_data.at[0, 'player2Name'] == None:
    print("player2Name was blank. Filling with 'fillInplayer2Name'")
    shot_data.at[0, 'player2Name'] = fillInPlayer2Name
else:
    print("Check Passed: player2Name is not blank. Current value:", shot_data.at[0, 'player2Name'])


### Error Check: 
#### Fills in PointNumber if not there

In [ ]:
 # Check if both conditions passed
if 'pointNumber' in shot_data.columns and not shot_data['pointNumber'].isnull().any() and shot_data['pointNumber'].is_monotonic_increasing:
    print("Check Passed: Point Numbers already exist")


# Check if pointNumber exists for every row and is not empty
if 'pointNumber' not in shot_data.columns or shot_data['pointNumber'].isnull().any():
    # Execute the script to calculate pointNumber
    point_starts = (shot_data['isPointStart'] == 1)
    shot_data['pointNumber'] = point_starts.cumsum()
    
    print("Data had missing point numbers. They were filled in automatically")
    
# Check if pointNumber is in increasing order
if not shot_data['pointNumber'].is_monotonic_increasing:
    # Print the condition failure if the 'pointNumber' column is not in increasing order
    print("Data had point numbers in the wrong order. They were filled in automatically")
    point_starts = (shot_data['isPointStart'] == 1)
    shot_data['pointNumber'] = point_starts.cumsum()
   

In [ ]:
missing_pointNumber_rows = shot_data[shot_data['pointNumber'].isnull()]

# Check if there are missing rows
if len(missing_pointNumber_rows) == 0:
    print("Check Passed: All rows have pointNumber")
else:
    print(f"Count of rows missing 'pointNumber': {len(missing_pointNumber_rows)}")

### Error Check: 
#### Check for NA's

- all 0, except missing pointScore should match amount of tiebreak shots
- side can have missing, for old tagger and not tagging side of each shot

### Fixing:
- Open CSV in Google Sheets (Excel will change to date format), output game, set, point Score of missing values. Then find in google sheets, adjust, redownload, and re-upload into notebook

In [ ]:
# Count empty strings in each column
empty_string_counts = (shot_data == "").sum()

# Filter out columns with zero empty strings
non_zero_counts = empty_string_counts[empty_string_counts > 0]

# Count NaN values in 'pointScore' column when 'gameScore' is '6-6'
na_tiebreak_count = shot_data.loc[shot_data['gameScore'] == '6-6', 'pointScore'].isna().sum()

side_na_count = shot_data.loc[shot_data['isPointStart'] == 1, 'side'].isna().sum()
print("\nCount of NaN values for 'side' on Point Start:", side_na_count)

# Display the count
print("\nCount of Nan when gameScore is '6-6' (# tiebreak shots):", na_tiebreak_count)


# Count NaN or empty values in specified columns
na_counts = shot_data[[ 'pointScore', 'shotInRally', 'gameScore', 'setScore', 'side', 'serverName']].isna().sum()

# Display the counts
print(f"\nCout of NA's in these columns\n{na_counts}")

# Display the counts
print("Count of empty strings in each column that includes at least one:")
print(non_zero_counts)



### Error Check: 
####  rows where side is empty
- If side missing, focus on rows where (shotInRally == 1) and returns (shotInRally == 2 ), anything else can leave NA

In [ ]:
print(shot_data[shot_data["side"].isnull()][['pointScore', 'gameScore', 'setScore','side', 'shotInRally']])

### Error Check: 
####  Outputs Missing shotInRally Rows

In [ ]:
empty_shot_rows = shot_data[shot_data['shotInRally'].isnull()]

if not empty_shot_rows.empty:
    # Iterate over the index of empty_shot_rows
    for index in empty_shot_rows.index:
        # Get the row with empty shotInRally
        empty_row = shot_data.loc[index]
        
        # Get the row above it
        if index - 1 >= 0:
            above_row = shot_data.loc[index - 1]
        else:
            above_row = None
        
        # Get the row below it
        if index + 1 < len(shot_data):
            below_row = shot_data.loc[index + 1]
        else:
            below_row = None
        
        # Print the rows
        print("Empty row:")
        print(empty_row)
        print("Row above:")
        print(above_row)
        print("Row below:")
        print(below_row)
        print("====================")
else:
    print("Check Passed: No Missing shotInRally")


### Error Check: 
####  Check that pointNumber is increasing consecutively

In [ ]:
point_numbers = shot_data['pointNumber'].unique()

# Check if the point numbers are consecutive
if list(point_numbers) == list(range(1, len(point_numbers) + 1)):
    print("Check Passed")
else:
    print("Error: The 'pointNumber' column does not contain consecutive numbers starting from 1.")


### Error Check: 
#### Check same amount of start and end points

In [ ]:
# Count of Point Start and Point End
num_point_start = shot_data['isPointStart'].sum()

# Count the number of rows where isPointEnd is equal to 1
num_point_end = shot_data['isPointEnd'].sum()

print("Number of rows with isPointStart = 1:", num_point_start)
print("Number of rows with isPointEnd = 1:", num_point_end)

### Error Check: 
#### Count of Missing Start Time, Output Rows

In [ ]:
# Find rows where isPointStart is True, but missing pointStartTime
missing_start_time = shot_data[((shot_data['isPointStart'] == 1) & shot_data['pointStartTime'].isna())]
print(f"Count of rows where PointStartTime is missing: {len(missing_start_time)}")


print(f"These are rows where PointStartTime is missing \n{missing_start_time}")

### Error Check: 
#### Count of Missing End Time, Output Rows

In [ ]:
# Find rows where isPointEnd is True, but missing pointEndTime
missingEndTime = shot_data[((shot_data['isPointEnd'] == 1) & shot_data['pointEndTime'].isna())]
print(f"Count of rows where PointEndTime is missing: {len(missingEndTime)}")

# Display the missing rows 
print(f"These are rows where PointEndTime is missing \n{missingEndTime}")

### Error Check: 
#### Make Jan-00 back into 1-0 for Game/Set Score

In [ ]:
# Make Scores Strings not Date Time
columns_to_convert = ['gameScore', 'setScore'] #if no tiebreakScore
# columns_to_convert = ['gameScore', 'setScore', 'tiebreakScore']
shot_data[columns_to_convert] = shot_data[columns_to_convert].astype(object)

In [ ]:
import re

# Define a mapping for month abbreviations
month_mapping = {'Jan': '1', 'Feb': '2', 'Mar': '3', 'Apr': '4', 'May': '5', 'Jun': '6',
                 'Jul': '7', 'Aug': '8', 'Sep': '9', 'Oct': '10', 'Nov': '11', 'Dec': '12'}

# Function to convert string like 'Jan-00' to '1-0'
def convert_score_string(score_str):
    # Check if the string has a month abbreviation and a year ending with '00'
    if re.match(r'^\d{1,2}-[A-Za-z]{3}$', score_str):
        # Extract year and month abbreviation
        year, month = score_str.split('-')

        # Remove leading zeros from the year
        year = str(int(year))

        # Replace month abbreviation with corresponding number
        month_number = month_mapping.get(month, month)

        # Concatenate the parts to form the transformed string
        transformed_str = f'{year}-{month_number}'

        return transformed_str

    # Check if the string has a month abbreviation and a year with leading '0's
    elif re.match(r'^[A-Za-z]{3}-\d{1,2}$', score_str):
        # Extract month abbreviation and year
        month, year = score_str.split('-')

        # Replace month abbreviation with corresponding number
        month_number = month_mapping.get(month, month)

        # Remove leading zeros from the year
        year = str(int(year))

        # Concatenate the parts to form the transformed string
        transformed_str = f'{month_number}-{year}'

        return transformed_str

    return score_str

# Apply the conversion function to the relevant columns in shot_data
shot_data['gameScore'] = shot_data['gameScore'].apply(convert_score_string)
shot_data['setScore'] = shot_data['setScore'].apply(convert_score_string)
# shot_data['tiebreakerScore'] = shot_data['tiebreakerScore'].apply(convert_score_string)


### Error Check: 
#### Check that there is no strings in set, game scores

In [ ]:
# Assuming shot_data is your DataFrame
unique_set_scores = shot_data['setScore'].unique()
unique_game_scores = shot_data['gameScore'].unique()

# Print unique values
print("Unique Set Scores:", unique_set_scores)
print("Unique Game Scores:", unique_game_scores)


### Error Check: 
#### Output consecutive isPointStart with no isPointEnd, and vice versa for isPointEnd

In [ ]:
# Output total start, total end
# How many Points aren't enclosing between point start and point end 

# Count where isPointStart = 1
count_isPointStart = (shot_data['isPointStart'] == 1).sum()

# Count where isPointEnd = 1
count_isPointEnd = (shot_data['isPointEnd'] == 1).sum()

print("Total count where isPointStart = 1:", count_isPointStart)
print("Total count where isPointEnd = 1:", count_isPointEnd)



# Initialize variables to keep track of the state
has_start = False
overlapping_ends = []

# Iterate through the DataFrame
for i in range(len(shot_data)):
    if shot_data['isPointStart'][i] == 1:
        has_start = True
    elif shot_data['isPointEnd'][i] == 1:
        if not has_start:
            overlapping_ends.append(i)
        else:
            has_start = False

# Display the rows where isPointEnd doesn't have a corresponding isPointStart
print("Rows where isPointEnd doesn't have a corresponding isPointStart:")
print(shot_data.iloc[overlapping_ends])

# Initialize variables to keep track of the state
start_indices = []
consecutive_starts = []

# Iterate through the DataFrame
for i in range(len(shot_data) - 1):  # Iterate until the second last row
    if shot_data['isPointStart'][i] == 1 and shot_data['isPointEnd'][i] != 1:
        # Check if the next row also has isPointStart without a corresponding isPointEnd
        if shot_data['isPointStart'][i + 1] == 1 and shot_data['isPointEnd'][i + 1] != 1:
            consecutive_starts.append(i + 1)

# Display the rows where isPointStart may double up without a corresponding isPointEnd
print("Rows where isPointStart may double up without a corresponding isPointEnd:")
print(shot_data.iloc[consecutive_starts])


# Add More Columns

### Add Column: returnerName

In [ ]:
# Get Returner Name for Shot DF

# Extract player1Name and player2Name from the first row of shot_data
first_row_shot_df = shot_data.iloc[0]
player1_name = first_row_shot_df['player1Name']
player2_name = first_row_shot_df['player2Name']

# # Can Also Manually inpute Player 1 and 2 name
# player1_name = "Kimmi Hance"
# player2_name = "Malaika Rapolu"

def get_returner_name(server_name):
    return player2_name if server_name == player1_name else player1_name

# Add 'Returner Name' column to point_df using the function
shot_data['returnerName'] = shot_data['serverName'].apply(get_returner_name)

print(f"Player 1 = {player1_name}, Player 2 = {player2_name}")
# print(shot_data[['serverName','returnerName']])

### Add Column: shotHitBy

In [ ]:
# Add Shot Hit By for each Shot

shot_data['shotHitBy'] = shot_data.apply(lambda row: row['serverName'] if row['shotInRally'] % 2 == 1 else row['returnerName'], axis=1)
# print(shot_data[['serverName', 'shotHitBy','pointNumber', 'shotInRally']].head(20))

### Add Column: player1Hand, player2Hand

In [ ]:
# Change this with whatever is player1Hand and player2Hand first Column Values
fill_in_player1hand = "Right"
fill_in_player2hand = "Right"

# Find the first non-null value in player1Hand column
player1hand = shot_data['player1Hand'].dropna().iloc[0] if 'player1Hand' in shot_data.columns else fill_in_player1hand
player2hand = shot_data['player2Hand'].dropna().iloc[0] if 'player2Hand' in shot_data.columns else fill_in_player2hand

# Display the updated values
print("player1Hand:", player1hand)
print("player2Hand:", player2hand)

shot_data['player1Hand'] = player1hand
shot_data['player2Hand'] = player2hand

### Add Column: isInsideOut, isInsideIn

In [ ]:
# Add columns for isInsideOut and isInsideIn, initially set to 0
shot_data['isInsideOut'] = None
shot_data['isInsideIn'] = None

# Iterate through rows
for index, row in shot_data.iterrows():
    shotHitBy = row['shotHitBy']
    player_hand = player1hand if shotHitBy == row['player1Name'] else player2hand
    
    if player_hand == "Right":
        if row['side'] == "Deuce" and row['shotFhBh'] == "Backhand" and row['shotDirection'] == "Crosscourt":
            shot_data.at[index, 'isInsideOut'] = 1
        elif row['side'] == "Ad" and row['shotFhBh'] == "Forehand" and row['shotDirection'] == "Crosscourt":
            shot_data.at[index, 'isInsideOut'] = 1
        elif row['side'] == "Deuce" and row['shotFhBh'] == "Backhand" and row['shotDirection'] == "Down the Line":
            shot_data.at[index, 'isInsideIn'] = 1
        elif row['side'] == "Ad" and row['shotFhBh'] == "Forehand" and row['shotDirection'] == "Down the Line":
            shot_data.at[index, 'isInsideIn'] = 1
    elif player_hand == "Left":
        if row['side'] == "Ad" and row['shotFhBh'] == "Backhand" and row['shotDirection'] == "Crosscourt":
            shot_data.at[index, 'isInsideOut'] = 1
        elif row['side'] == "Deuce" and row['shotFhBh'] == "Forehand" and row['shotDirection'] == "Crosscourt":
            shot_data.at[index, 'isInsideOut'] = 1
        elif row['side'] == "Ad" and row['shotFhBh'] == "Backhand" and row['shotDirection'] == "Down the Line":
            shot_data.at[index, 'isInsideIn'] = 1
        elif row['side'] == "Deuce" and row['shotFhBh'] == "Forehand" and row['shotDirection'] == "Down the Line":
            shot_data.at[index, 'isInsideIn'] = 1

### Add Column: isAce

In [ ]:
# Add the Ace column
shot_data['isAce'] = None

for index, row in shot_data.iterrows():
    if row['isPointEnd'] == 1:
        if row['shotInRally'] == 1: # last point is serve
            if (row['firstServeIn'] == 1 or row['secondServeIn'] == 1): # either first or second serve went in
                shot_data.at[index, 'isAce'] = 1

### Add Column: isDoubleFault

In [ ]:
# Add the DoubleFault column
shot_data['isDoubleFault'] = None

for index, row in shot_data.iterrows():
    if row['isPointEnd'] == 1:
        if row['shotInRally'] == 1: # last point is serve
            if (row['firstServeIn'] != 1 and row['secondServeIn'] != 1): # either first or second serve went in
                shot_data.at[index, 'isDoubleFault'] = 1

### Add Column: pointWonBy, lastShotError

In [ ]:
# Add the 'pointWonBy' column
shot_data['pointWonBy'] = None

# Add the 'lastShotError' column
shot_data['lastShotError'] = 0

for index, row in shot_data.iterrows():
    if row['isPointEnd'] == 1:
        if row['shotInRally'] == 1: # last point is serve
            if row['isAce'] == 1: 
                shot_data.at[index, 'pointWonBy'] = row['serverName']
            elif row['isDoubleFault'] == 1: 
                shot_data.at[index, 'pointWonBy'] = row['returnerName']

                
        elif row['shotInRally'] != 1:
            if row['isErrorWideR'] == 1 or row['isErrorWideL'] == 1 or row['isErrorNet'] == 1 or row['isErrorLong'] == 1: # if error
                shot_data.at[index, 'lastShotError'] = 1
                
                if row['shotInRally'] % 2 == 0:
                    shot_data.at[index, 'pointWonBy'] = row['serverName']
                else:
                    shot_data.at[index, 'pointWonBy'] = row['returnerName']
        
            elif row['isWinner'] == 1:
                if row['shotInRally'] % 2 == 0:
                    shot_data.at[index, 'pointWonBy'] = row['returnerName']
                else:
                    shot_data.at[index, 'pointWonBy'] = row['serverName']


### Error Check: 
#### Output Point End with no pointWonBy


In [ ]:

# Missing Point End
print(shot_data[(shot_data['isPointEnd'] == 1) & (shot_data['pointWonBy'].isnull())][['pointScore', 'gameScore','setScore', 'lastShotError', 'isWinner', 'isErrorWideR', 'isErrorWideL',
       'isErrorNet', 'isErrorLong', 'pointWonBy', 'serverName', 'shotInRally']])


### Add Column: serveResult, serveInPlacement

In [ ]:
conditions = [
    (shot_data['isPointStart'] == 1) & (shot_data['firstServeIn'] == 1),
    (shot_data['isPointStart'] == 1) & (shot_data['firstServeIn'] != 1) & (shot_data['secondServeIn'] == 1),
    (shot_data['isPointStart'] == 1) & (shot_data['firstServeIn'] != 1) & (shot_data['secondServeIn'] != 1)
]

# Define the values to be assigned for each condition
values_result = ['1st Serve In', '2nd Serve In', 'Double Fault']
values_placement = [shot_data['firstServeZone'], shot_data['secondServeZone'], np.nan]

# Use numpy.select to assign values based on conditions
shot_data['serveResult'] = np.select(conditions, values_result, default='')
shot_data['serveInPlacement'] = np.select(conditions, values_placement, default='')

# Now shot_data DataFrame is updated with serveResult and serveInPlacement values where isPointStart is 1
# print(shot_data[['serveResult', 'serveInPlacement','pointScore','side']].head(60))

### Filling Columns: Forward Fill
- 'gameScore'
- 'setScore'
- 'serverName'
- 'player1Name',
- 'player2Name'
- 'player1Hand'
- 'player2Hand'

In [ ]:
# Forward Fill GameScore, SetScore, tiebreakScore, serverName, player1Name, player2Name

columns_to_fill = ['gameScore','setScore', 'serverName', 'player1Name', 'player2Name', 'player1Hand', 'player2Hand'] 
# columns_to_fill = ['gameScore','setScore', 'tiebreakScore', 'serverName', 'player1Name', 'player2Name'] 
# Add player1Hand, player2Hand when it is in tagger

for column in columns_to_fill:
    shot_data[column].replace(['', 'na'], pd.NaT, inplace=True)
    shot_data[column] = shot_data[column].ffill()

In [ ]:
shot_data.replace("", None, inplace=True)

# Output ShotCSV

In [ ]:
# # Ouput Improved Shot Csv HERE
# Assuming point_df is your DataFrame and player1Name and player2Name are the names from the first row
player1NameNoSpace = shot_data.iloc[0]['player1Name'].replace(" ", "")
player2NameNoSpace = shot_data.iloc[0]['player2Name'].replace(" ", "")

# Save DataFrame to CSV file with modified player names
shot_data.to_csv(f'Shot_Visuals_{player1NameNoSpace}_{player2NameNoSpace}.csv', index=False)

In [ ]:
shot_data['pointNumber'].unique()

# Below is for Point CSV

### Create Point DF

In [ ]:
# Creating point_df (with only 1 row for each pointNumber)
point_df = shot_data.drop_duplicates(subset='pointNumber')[['pointNumber']]
# point_df.shape

### Add Column: player1Name, player2Name
### Set Variable: first_player1Name, first_player2Name

In [ ]:
# Extract the first value of player1Name and player2Name from shot_data
first_player1Name = shot_data['player1Name'].iloc[0]
first_player2Name = shot_data['player2Name'].iloc[0]

# Fill in the first value into all rows of point_df['player1Name'] and point_df['player2Name']
point_df['player1Name'] = first_player1Name
point_df['player2Name'] = first_player2Name

### Add Column: Scores

In [ ]:
point_df['pointScore'] = shot_data.groupby('pointNumber')['pointScore'].first().values
point_df['gameScore'] = shot_data.groupby('pointNumber')['gameScore'].first().values
point_df['setScore'] = shot_data.groupby('pointNumber')['setScore'].first().values
point_df['tiebreakScore'] = shot_data.groupby('pointNumber')['tiebreakScore'].first().values

### Add Column: Side

In [ ]:
# Group shot_data by 'pointNumber' and get the first 'side' value for each group
side_values = shot_data.groupby('pointNumber')['side'].first().reset_index()

point_df['side'] = side_values['side'].values

### Add Column: serverName, returnerName

In [ ]:
# Adds Server and Returner Names and pointScore

point_df['serverName'] = shot_data.groupby('pointNumber')['serverName'].first().values
point_df['returnerName'] = shot_data.groupby('pointNumber')['returnerName'].first().values


## Warning: Will be Empty if Timestamp separate
### Add Column: serverName, returnerName

In [ ]:
# Add Start and End times per point

for index, row in shot_data.iterrows():
    point_number = row['pointNumber']
    
    if row['isPointStart'] == 1:
        point_df.loc[point_df['pointNumber'] == point_number, 'Position'] = row['pointStartTime']
    if row['isPointEnd'] == 1:
        point_df.loc[point_df['pointNumber'] == point_number, 'pointEndPosition'] = row['pointEndTime']

# Add Duration
point_df['Duration'] = point_df['pointEndPosition'] - point_df['Position']

### Add Column: Rally Column
- rallyCount (x amount)
- rallyCountFreq (1-4,5-8,9-12,13+)

In [ ]:
# Find the highest shotInRally for each pointNumber in shot_data
max_rally_per_point = shot_data.groupby('pointNumber')['shotInRally'].max().reset_index()
point_df['rallyCount'] = list(max_rally_per_point['shotInRally'])

# Add 'rallyCountFreq' column based on specified conditions
point_df['rallyCountFreq'] = point_df['rallyCount'].apply(lambda x: '1 - 4' if 1 <= x <= 4 else ('5 - 8' if 5 <= x <= 8 else ('9 - 12' if 9 <= x <= 12 else ('13 +' if x >= 13 else 'Error'))))


### Add Column: Serve 
- firstServeIn
- secondServeIn
- serveResult
- serveInPlacement

Part 2
- serveResult
- serveInPlacement
 
Part 3
- firstServeZone
- secondServeZone

In [ ]:
# Add firstServeIn and secondServeIn

# Add firstServeIn and secondServeIn columns
point_df['firstServeIn'] = 0
point_df['secondServeIn'] = 0

for point_number in shot_data['pointNumber'].unique():
    # Check if firstServeIn is 1 for the given pointNumber in shot_data
    if any((shot_data['pointNumber'] == point_number) & (shot_data['firstServeIn'] == 1)):
        point_df.loc[point_df['pointNumber'] == point_number, 'firstServeIn'] = 1
    
    # Check if secondServeIn is 1 for the given pointNumber in shot_data
    if any((shot_data['pointNumber'] == point_number) & (shot_data['secondServeIn'] == 1)):
        point_df.loc[point_df['pointNumber'] == point_number, 'secondServeIn'] = 1

#part 2
start_points = shot_data[shot_data['isPointStart'] == 1]

# Set values in point_df to corresponding values from start_points
point_df['serveResult'] = start_points['serveResult'].values
point_df['serveInPlacement'] = start_points['serveInPlacement'].values

# part 3
serve_zones = shot_data.loc[shot_data['shotInRally'] == 1, ['pointNumber', 'firstServeZone', 'secondServeZone', 'firstServeIn', 'secondServeIn']].drop_duplicates()
point_df['firstServeZone'] = shot_data.groupby('pointNumber')['firstServeZone'].first().values
point_df['secondServeZone'] = shot_data.groupby('pointNumber')['secondServeZone'].first().values

### Add Column: Ace

In [ ]:
point_df['isAce'] = ((point_df['rallyCount'] == 1) & ((point_df['serveResult'] != "Double Fault")))

# Display the resulting DataFrame
print(point_df[['pointNumber', 'rallyCount', 'isAce']].head(14))

### Add Column: Return

In [ ]:
# print(shot_data[shot_data['shotInRally'] == 2][['shotDirection','shotFhBh','pointScore','gameScore','setScore']])

In [ ]:
# Set the initial values of 'returnDirection' and 'returnHand' columns to None
point_df['returnDirection'] = None
point_df['returnFhBh'] = None

# Iterate through pointNumber in shot_data
for point_number in shot_data['pointNumber'].unique():
    # Check if shotInRally == 2 exists for the given pointNumber
    if 2 in shot_data.loc[shot_data['pointNumber'] == point_number, 'shotInRally'].values:
        # Get the information from the corresponding row
        row_with_return_info = shot_data[(shot_data['pointNumber'] == point_number) & (shot_data['shotInRally'] == 2)].iloc[0]

        # Assign values to 'returnDirection' and 'returnHand' columns
        point_df.loc[point_df['pointNumber'] == point_number, 'returnDirection'] = row_with_return_info['shotDirection']
        point_df.loc[point_df['pointNumber'] == point_number, 'returnFhBh'] = row_with_return_info['shotFhBh']

# Display the modified DataFrame
print(point_df.head(10))

### Add Column: Error Column
- errorType

In [ ]:
# Create an empty DataFrame to store the results
error_results = pd.DataFrame(columns=['errorType', 'pointNumber'])

# Iterate through entire shot_data
for index, row in shot_data.iterrows():
    pointNumber = row['pointNumber']
    point_error_value = None
    
    if row['isErrorWideR'] == 1:
        point_error_value = 'Wide Right'
    elif row['isErrorWideL'] == 1:
        point_error_value = 'Wide Left'
    elif 'isErrorNet' in row and row['isErrorNet'] == 1:
        point_error_value = 'Net'
    elif row['isErrorLong'] == 1:
        point_error_value = 'Long'
    

    # If an error is found, append the result to the error_results DataFrame
    if point_error_value is not None:
        error_results = pd.concat([error_results, pd.DataFrame({'pointNumber': [pointNumber], 'errorType': [point_error_value]})], ignore_index=True)


# Drop duplicates based on 'pointNumber'
error_results = error_results.drop_duplicates(subset=['pointNumber'])

In [ ]:
# Create a dictionary mapping 'pointNumber' to 'errorType' in error_results
error_type_mapping = dict(zip(error_results['pointNumber'], error_results['errorType']))

# Create 'errorType' column in point_df based on the mapping
point_df['errorType'] = point_df['pointNumber'].map(error_type_mapping)

point_df = point_df.replace({np.nan: None})

### Add Column: Error (Returns)

In [ ]:
def get_return_error(row):
    if row['rallyCount'] == 2:
        return row['errorType']
    else:
        return None

point_df.loc[point_df['pointNumber'] == point_number, 'serveInPlacement'] = shot_data['secondServeZone']  

# Apply the functions to create the new columns
point_df['returnError'] = point_df.apply(get_return_error, axis=1)


In [ ]:
# print(point_df["returnError"].unique())
# print(point_df[['rallyCount','returnDirection','returnFhBh']])

### Add Column: Last Shot

In [ ]:
point_df['lastShotDirection'] = None
point_df['lastShotFhBh'] = None
point_df['lastShotHitBy'] = None  
point_df['lastShotResult'] = None  

# Iterate through unique pointNumbers in shot_data
for point_number in shot_data['pointNumber'].unique():
    # Check if isPointEnd == 1 exists for the given pointNumber
    if 1 in shot_data.loc[shot_data['pointNumber'] == point_number, 'isPointEnd'].values:
        # Get the information from the corresponding row
        row_with_lastshot_info = shot_data[(shot_data['pointNumber'] == point_number) & (shot_data['isPointEnd'] == 1)].iloc[0]

        # Assign values to 'lastShotDirection' and 'lastShotFhBh' columns
        point_df.loc[point_df['pointNumber'] == point_number, 'lastShotDirection'] = row_with_lastshot_info['shotDirection']
        point_df.loc[point_df['pointNumber'] == point_number, 'lastShotFhBh'] = row_with_lastshot_info['shotFhBh']
        point_df.loc[point_df['pointNumber'] == point_number, 'lastShotHitBy'] = row_with_lastshot_info['shotHitBy']
        
        # Determine lastShotResult based on conditions
        if row_with_lastshot_info['isWinner'] == 1 and not row_with_lastshot_info['isAce']:
            point_df.loc[point_df['pointNumber'] == point_number, 'lastShotResult'] = "Winner"
        elif row_with_lastshot_info['lastShotError'] == 1:
            point_df.loc[point_df['pointNumber'] == point_number, 'lastShotResult'] = "Error"

In [ ]:
# # Display the modified DataFrame
# print(point_df[['rallyCount','lastShotDirection','lastShotFhBh', 'lastShotHitBy']])

In [ ]:
print(point_df['lastShotResult'].unique())

### Add Column: pointWonBy

In [ ]:
# Initialize variables to keep track of the state
prev_point_number = None
point_won_by_list = []

# Iterate through the DataFrame
for index, row in shot_data.iterrows():
    if row['isPointEnd'] == 1:
        # Check if pointNumber is different and consecutively increasing
        if prev_point_number is None or row['pointNumber'] == prev_point_number + 1:
            # Append pointWonBy to the list
            point_won_by_list.append(row['pointWonBy'])
            prev_point_number = row['pointNumber']
        else:
            print("Error: Point numbers are not different or consecutively increasing.")
            break

# Add point_won_by_list as a new column to point_df
point_df['pointWonBy'] = point_won_by_list

### Error Check: 
#### player1, player2, serverName, returnerName do not have misspellings

In [ ]:
print(point_df['player1Name'].unique())
print(point_df['player2Name'].unique())
print(point_df['serverName'].unique())
print(point_df['returnerName'].unique())

### Error Check: 
#### pointWonBy has value that is not one of the two player names
- References: first_player1Name, first_player2Name from above server info in point_df

In [ ]:
print(point_df['pointWonBy'].unique())
none_pointWonBy_df = point_df[~point_df['pointWonBy'].isin([first_player1Name, first_player2Name])]

print(none_pointWonBy_df)

### Add Column: Break Point

In [ ]:
# List of values to check for in 'Name' column
break_point_values = ['0-40', '15-40', '30-40', '40-40 (Deuce)', '40-40 (Ad)']

# Create 'isBreakPoint' column in point_df
point_df['isBreakPoint'] = point_df['pointScore'].isin(break_point_values)

## Warning: Do not use if atNetPlayer1 is in shot_data
### Add Column: atNetPlayer1, atNetPlayer2

- if below says atNetPlayer1: Uncomment below Code
- replace later with atNetPlayer1 atNetPlayer2

In [ ]:
# Check if 'atNetPlayer1' exists in point_df
if 'atNetPlayer1' in shot_data.columns:
    print("Column 'atNetPlayer1' exists in point_df.")
else:
    print("Column 'atNetPlayer1' does not exist in point_df.")

# Uncomment if atNetPlayer1 does not exist
import numpy as np

# Copy the DataFrame
at_net_df2 = shot_data.copy()

# Record values for specified columns
at_net_df2 = at_net_df2[['player1Name', 'player2Name', 'isVolley', 'serverName', 'shotHitBy', 'returnerName', 'pointNumber', 'shotInRally']].copy()

# Add columns for aggregation
at_net_df2['atNetPlayer1'] = np.where((at_net_df2['isVolley'] == 1) & (at_net_df2['shotHitBy'] == at_net_df2['player1Name']), 1, 0)
at_net_df2['atNetPlayer2'] = np.where((at_net_df2['isVolley'] == 1) & (at_net_df2['shotHitBy'] == at_net_df2['player2Name']), 1, 0)

# Aggregate atNetPlayer1 and atNetPlayer2 based on pointNumber
at_net_df2['atNetPlayer1_Agg'] = at_net_df2.groupby('pointNumber')['atNetPlayer1'].transform('max')
at_net_df2['atNetPlayer2_Agg'] = at_net_df2.groupby('pointNumber')['atNetPlayer2'].transform('max')

# Drop the duplicate rows created during aggregation
at_net_df2.drop_duplicates(subset=['pointNumber'], inplace=True)

point_df['atNetPlayer1'] = at_net_df2['atNetPlayer1_Agg'].values
point_df['atNetPlayer2'] = at_net_df2['atNetPlayer2_Agg'].values

### Add Column: setNum

In [ ]:
# Extract numbers from 'setScore' and calculate the sum plus 1
point_df['setNum'] = point_df['setScore'].apply(lambda x: sum(int(char) for char in x if char.isdigit()) + 1)

In [ ]:
point_df.columns

### Add Column: Game Number, Set Number, Game/Set/Point for each player

In [ ]:
point_df[['player1SetScore', 'player2SetScore']] = point_df['setScore'].str.split('-', expand=True)
point_df[['player1GameScore', 'player2GameScore']] = point_df['gameScore'].str.split('-', expand=True)
point_df[['player1PointScore', 'player2PointScore']] = point_df['pointScore'].str.split('-', expand=True)
if not point_df['tiebreakScore'].isnull().all() and not point_df['tiebreakScore'].eq("").all():
    # Perform the operation only when tiebreakScore is not empty
    point_df[['player1TiebreakScore', 'player2TiebreakScore']] = point_df['tiebreakScore'].str.split('-', expand=True)
else:
    # Set player1TiebreakScore and player2TiebreakScore to NaN
    point_df['player1TiebreakScore'] = np.nan
    point_df['player2TiebreakScore'] = np.nan
    
def calculate_game_number(score):
    return int(score.split('-')[0]) + int(score.split('-')[1])+1

# Apply the function to create the 'gameNumber' column
point_df['gameNumber'] = point_df['gameScore'].apply(calculate_game_number)

### Error Check: 
#### Output Game Number in order: Should be consecutive increasing. Ex: 1,2,3,4,5,6. End of Set 1. 1,2,3,4,5,6,7,8

In [ ]:
game_numbers = point_df['gameNumber'].tolist()

# Initialize variables
seen = set()
prev = None

# Iterate through gameNumber column
for num in game_numbers:
    # If the number is not in seen or it's different from the previous one, print it
    if num not in seen or num != prev:
        print(num, end=', ')
    # If the number is the same as the previous one but not consecutive, print it
    elif num == prev and num not in seen:
        print(num, end=', ')
    # Update seen set and prev variable
    seen.add(num)
    prev = num

### Add Column: player1ServeResult

In [ ]:
# Add the 'player1ServeResult' column
point_df['player1ServeResult'] = None

# Set player1ServeResult based on conditions
point_df.loc[point_df['serverName'] == point_df['player1Name'], 'player1ServeResult'] = point_df['serveResult']

In [ ]:
point_df.loc[point_df['isAce'] == True, 'player1ServeResult'] = 'Ace'


In [ ]:
# print(point_df['serveInPlacement'].head(10))

### Add Column: player1ServePlacement

In [ ]:
# Add the 'player1ServePlacement' column
point_df['player1ServePlacement'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['serverName'] == point_df['player1Name'], 'player1ServePlacement'] = point_df['side'] + ': ' + point_df['serveInPlacement']

### Add Column: player1ReturnPlacement

In [ ]:
# Add the 'player1ReturnPlacement' column
point_df['player1ReturnPlacement'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['returnerName'] == point_df['player1Name'], 'player1ReturnPlacement'] = point_df['returnDirection']

### Add Column: player1ReturnFhBh

In [ ]:
# Add the 'player1ReturnFhBh' column
point_df['player1ReturnFhBh'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['returnerName'] == point_df['player1Name'], 'player1ReturnFhBh'] = point_df['returnFhBh']

### Add Column: player1LastShotPlacement

In [ ]:
# Add the 'player1LastShotFhBh' column
point_df['player1LastShotPlacement'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['lastShotHitBy'] == point_df['player1Name'], 'player1LastShotPlacement'] = point_df['lastShotDirection']

### Add Column: player1LastShotFhBh

In [ ]:
# Add the 'player1LastShotFhBh' column
point_df['player1LastShotFhBh'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['lastShotHitBy'] == point_df['player1Name'], 'player1LastShotFhBh'] = point_df['lastShotFhBh']

### Add Column: player1LastShotResult

In [ ]:
# Add the 'player1LastShotResult' column
point_df['player1LastShotResult'] = None

# Set player1LastShotResult based on conditions, excluding 'Ace' and 'Double Fault'
point_df.loc[
    (point_df['lastShotHitBy'] == point_df['player1Name']) & 
    ~point_df['player1ServeResult'].isin(['Ace', 'Double Fault']), 
    'player1LastShotResult'
] = point_df['lastShotResult']


In [ ]:
print(point_df[['setNum', 'gameScore','pointScore','player1ServePlacement','serveInPlacement']].head(10))

## Add Column: Name (For Video)

In [ ]:
# Change pointScore to the specified format
point_df['Name'] = point_df.apply(lambda row: f"Set {row['setNum']}: {row['gameScore']}, {row['tiebreakScore']} {row['serverName']} Serving" if pd.notna(row['tiebreakScore']) else f"Set {row['setNum']}: {row['gameScore']}, {row['pointScore']} {row['serverName']} Serving", axis=1)


In [ ]:
print(point_df[['setNum', 'gameScore','pointScore','tiebreakScore','Name']].head(5))
print(point_df[['setNum', 'gameScore','pointScore','tiebreakScore','Name']].tail(5))

## Reorder DataFrame for Output

In [ ]:
point_df_copy = point_df.copy()

In [ ]:
point_df.shape

In [ ]:
point_df_copy.shape

In [ ]:
desired_order = [
    'Name', 'pointNumber', 'setNum', 'gameNumber', 'player1Name', 'player2Name',
    'pointScore', 'gameScore', 'setScore', 'tiebreakScore', 'side', 'serverName',
    'returnerName', 'Position', 'pointEndPosition', 'Duration', 'pointWonBy',
    'rallyCount', 'rallyCountFreq', 'firstServeIn', 'secondServeIn',
    'serveResult', 'serveInPlacement', 'firstServeZone', 'secondServeZone',
    'isAce', 'returnDirection', 'returnFhBh', 'errorType', 'returnError',
    'lastShotDirection', 'lastShotFhBh', 'lastShotHitBy', 'lastShotResult',
    'isBreakPoint', 'atNetPlayer1', 'atNetPlayer2', 'player1SetScore',
    'player2SetScore', 'player1GameScore', 'player2GameScore', 'player1PointScore',
    'player2PointScore', 'player1TiebreakScore', 'player2TiebreakScore',
    'player1ServeResult', 'player1ServePlacement', 'player1ReturnPlacement',
    'player1ReturnFhBh', 'player1LastShotPlacement', 'player1LastShotFhBh',
    'player1LastShotResult'
]

# Reorder the columns
point_df = point_df.reindex(columns=desired_order)

### Error Check: 
#### Check if the columns and their order are the same

In [ ]:
print(point_df.shape)
print(point_df_copy.shape)

In [ ]:
# Get the set of column names for each DataFrame
point_df_columns = set(point_df.columns)
point_df_copy_columns = set(point_df_copy.columns)

# Find the column names unique to each DataFrame
unique_to_point_df = point_df_columns - point_df_copy_columns
unique_to_point_df_copy = point_df_copy_columns - point_df_columns

# Output the results
if unique_to_point_df:
    print("Columns unique to point_df:", unique_to_point_df)
else:
    print("All columns in point_df are also in point_df_copy")

if unique_to_point_df_copy:
    print("Columns unique to point_df_copy:", unique_to_point_df_copy)
else:
    print("All columns in point_df_copy are also in point_df")


In [ ]:
# Check if the columns and their order are the same
if all(point_df.columns == point_df_copy.columns):
    print("Check Passed: Same columns")
else:
    print("Check Failed: Different columns")

# Check if the columns contain the same data
if point_df.equals(point_df_copy):
    print("Check Passed: Same data")
else:
    print("Check Failed: Different data")


## Warning: All nan to ""
### Alterating Data

In [ ]:
# Change Na to "" Empty String

# Assuming point_df is your DataFrame
# Replace NaN, None, and NA values with empty strings in point_df
point_df.replace([pd.NA, None, pd.NaT, float('nan')], "", inplace=True)

# PRINT POINTCSV VISUALS

In [ ]:
# Save point_df to CSV file

# Assuming point_df is your DataFrame and player1Name and player2Name are the names from the first row
player1NameNoSpace = point_df.iloc[0]['player1Name'].replace(" ", "")
player2NameNoSpace = point_df.iloc[0]['player2Name'].replace(" ", "")

# Save DataFrame to CSV file with modified player names
point_df.to_csv(f'Point_Visuals_{player1NameNoSpace}_{player2NameNoSpace}.csv', index=False)


## Warning: Creates Copy of point_df called point_df_for_viewer

In [ ]:
point_df_for_viewer = point_df.copy()

## Warning: Removing missing start time points
### Removing Data

In [ ]:
# Assuming pointdf is a DataFrame

# Display rows where 'isPointStart' is equal to 1 and 'pointStartTime' is NaN in pointdf
missing_start_time = point_df_for_viewer[point_df_for_viewer['Position'].isna() | (point_df_for_viewer['Position'] == "")]
print("Rows with missing start time:")
print(missing_start_time)



In [ ]:
# Remove those rows from pointdf
point_df_for_viewer = point_df_for_viewer.drop(missing_start_time.index)

## Warning: Changing Value of atNetPlayer1 Column
### Alterating Data

In [ ]:
# Change Value Names for Match Viewer Output

# Assuming point_df is your DataFrame and player1Name is the name from the first row
player1Name = point_df_for_viewer.iloc[0]['player1Name']

# Replace values in the 'atNetPlayer1' column
point_df_for_viewer['atNetPlayer1'] = point_df_for_viewer['atNetPlayer1'].replace({0: "", 1: player1Name})

## Warning: All nan to ""
### Alterating Data

In [ ]:
# Change Na to "" Empty String

# Assuming point_df is your DataFrame
# Replace NaN, None, and NA values with empty strings in point_df
point_df_for_viewer.replace([pd.NA, None, pd.NaT, float('nan')], "", inplace=True)

# Print POINTCSV UPLOAD

In [ ]:
player1Name = point_df_for_viewer.iloc[0]['player1Name'].replace(" ", "")
player2Name = point_df_for_viewer.iloc[0]['player2Name'].replace(" ", "")

# Save DataFrame to CSV file with modified player names
point_df_for_viewer.to_csv(f'Point_Upload_Website_{player1Name}_{player2Name}.csv', index=False)
point_df_for_viewer.to_json(f'Point_Upload_Website_{player1Name}_{player2Name}.json', orient='records')

# REMINDER: AT THIS POINT, NEED TO APPEND Position Timestamps to POINT FOR UPLOAD

# EDA

## Shot CSV EDA

In [ ]:
# Can input CSV Directly here for statistics functions

# your_file_name = "filename.csv"
# shot_eda = pd.read_csv(your_file_name)

# if directly from notebook
shot_eda = shot_data.copy()

In [ ]:
player1Name = shot_eda.iloc[0]['player1Name']

# Filter shot_data based on the conditions
approach_data_player1 = shot_eda[(shot_eda['isApproach'] == 1) & (shot_eda['shotHitBy'] == player1Name)]

# Count the distinct pointNumbers
distinct_point_numbers = approach_data_player1['pointNumber'].nunique()

# Print the result
print(f"Number of Approach Shots hit by {player1Name}: {distinct_point_numbers}" )

# print(approach_data_player1)


## Point CSV EDA

In [ ]:
# Can input CSV Directly here for statistics functions

# your_file_name = "filename.csv"
# point_df = pd.read_csv(your_file_name)



In [ ]:
first_player1Name = point_df['player1Name'].iloc[0]



# Display the results
print(f"\nServe Results for {first_player1Name}:")

# Assuming point_df is your DataFrame
total_serves = len(point_df[point_df['serverName'] == first_player1Name])
first_serve_in_count = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['firstServeIn'] == 1)])
first_serve_won_count = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['firstServeIn'] == 1) & (point_df['pointWonBy'] == first_player1Name)])
percentage_first_serve_in = (first_serve_in_count / total_serves) * 100 if total_serves > 0 else 0
percentage_first_serve_won = (first_serve_won_count / first_serve_in_count) * 100 if first_serve_in_count > 0 else 0

second_serve_total_count = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['firstServeIn'] == 0)])
second_serve_in_count = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['firstServeIn'] == 0)& (point_df['secondServeIn'] == 1)])
second_serve_won_count = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['firstServeIn'] == 0)& (point_df['secondServeIn'] == 1) & (point_df['pointWonBy'] == first_player1Name)])
percentage_second_serve_in = (second_serve_in_count / second_serve_total_count) * 100 if second_serve_total_count > 0 else 0
percentage_second_serve_won = (second_serve_won_count / second_serve_in_count) * 100 if second_serve_in_count > 0 else 0



# Display the results
print("\nTotal Serves:", total_serves)
print("First Serve In (Count):", first_serve_in_count)
print("First Serve Won (Count):", first_serve_won_count)
print(f"First Serve In (%): {percentage_first_serve_in:.2f}%")
print(f"First Serve Won (%): {percentage_first_serve_won:.2f}%")

print("Second Serve In (Count):", second_serve_in_count)
print("Second Serve Total (Count):", second_serve_total_count)
print("Second Serve Won (Count):", second_serve_won_count)
print(f"Second Serve In (%): {percentage_second_serve_in:.2f}%")
print(f"Second Serve Won (%): {percentage_second_serve_won:.2f}%")

# Assuming point_df is your DataFrame
count_is_ace = (point_df[point_df['serverName'] == first_player1Name]['isAce']).sum()
count_is_double_fault = ((point_df['serverName'] == first_player1Name) & (point_df['serveResult'] == "Double Fault")).sum()

# Display the results
print("Ace (Count):", count_is_ace)
print("Double Fault (Count):", count_is_double_fault)

# Count of rows where serverName is equal to the first row of player1Name and pointWonBy is equal to the first row of player1Name
total_service_points_won = len(point_df[(point_df['serverName'] == first_player1Name) & (point_df['pointWonBy'] == first_player1Name)])
total_service_points_won_percentage = total_service_points_won / total_serves *100

# Display the results
print(f"Points Won on Serve (Count) {total_service_points_won}")

print(f"Points Won on Serve (%): {total_service_points_won_percentage:.2f}%")

# Assuming point_df is your DataFrame
return_points = point_df[(point_df['returnerName'] == first_player1Name) & (point_df['rallyCount'] >= 2)]

total_return = len(return_points)
returnMade = len(return_points[(return_points['rallyCount'] > 2) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error'))])
returnError = len(return_points[(return_points['lastShotResult'] == 'Error') & (return_points['rallyCount'] == 2)])
returnWinner = len(return_points[(return_points['lastShotResult'] == 'Winner') & (return_points['rallyCount'] == 2)])
returnMadePercentage = returnMade/total_return

returnWonByPlayer1 = len(return_points[return_points['pointWonBy'] == first_player1Name])
returnWonByPlayer1Percentage = returnWonByPlayer1 / total_return * 100 if total_return > 0 else 0

deuceReturnCount = len(return_points[return_points['side'] == 'Deuce'])
adReturnCount = len(return_points[return_points['side'] == 'Ad'])


deuceReturnMade = len(return_points[(return_points['side'] == 'Deuce') & ((return_points['rallyCount'] > 2) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error')))])
adReturnMade = len(return_points[(return_points['side'] == 'Ad') & ((return_points['rallyCount'] > 2) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error')))])

deuceReturnMadePercentage = deuceReturnMade/deuceReturnCount
adReturnMadePercentage = adReturnMade/adReturnCount

deuceReturnWonByPlayer1 = len(return_points[(return_points['side'] == 'Deuce') & (return_points['pointWonBy'] == first_player1Name) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error'))])
adReturnWonByPlayer1 = len(return_points[(return_points['side'] == 'Ad') & (return_points['pointWonBy'] == first_player1Name) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error'))])

deuceReturnWonByPlayer1Percentage = deuceReturnWonByPlayer1 / deuceReturnCount * 100 if deuceReturnCount > 0 else 0
adReturnWonByPlayer1Percentage = adReturnWonByPlayer1 / adReturnCount * 100 if adReturnCount > 0 else 0




print(f"\nReturn Results for {first_player1Name}:\n")

print("Total Return (Count):", total_return)
print("Return Won (Count):", returnWonByPlayer1)
print("Return Won (%):", returnWonByPlayer1Percentage)

print("\nReturn Made (Count):", returnMade)
print("Return Made (%):", returnMadePercentage)
print("Return Error (Count):", returnError)
print("Return Winner (Count):", returnWinner)

print("\nDeuce Return (Count):", deuceReturnCount)
print("Deuce Return Made (Count):", deuceReturnMade)
print("Deuce Return Made (%):", deuceReturnMadePercentage)
print("Deuce Return Won by Player1 (%):", deuceReturnWonByPlayer1Percentage)
print("Deuce Return Won by Player1 (Count):", deuceReturnWonByPlayer1)


print("\nAd Return (Count):", adReturnCount)
print("Ad Return Made (Count):", adReturnMade)
print("Ad Return Made (%):", adReturnMadePercentage)
print("Ad Return Won by Player1 (Count):", adReturnWonByPlayer1)
print("Ad Return Won by Player1 (%):", adReturnWonByPlayer1Percentage)

# Assuming return_points is your DataFrame
deuce_return_points = return_points[(return_points['side'] == 'Deuce') & (return_points['returnerName'] == first_player1Name) & (return_points['rallyCount'] >= 2)]

# Deuce Return Points Separated by returnFhBh
deuce_forehand_return_points = deuce_return_points[deuce_return_points['returnFhBh'] == 'Forehand']
deuce_backhand_return_points = deuce_return_points[deuce_return_points['returnFhBh'] == 'Backhand']


# Count for Deuce Return Points - Made
count_deuce_forehand_made = len(deuce_forehand_return_points[(deuce_forehand_return_points['rallyCount'] > 2) | ((deuce_forehand_return_points['rallyCount'] == 2) & (deuce_forehand_return_points['lastShotResult'] != 'Error'))])
count_deuce_backhand_made = len(deuce_backhand_return_points[(deuce_backhand_return_points['rallyCount'] > 2) | ((deuce_backhand_return_points['rallyCount'] == 2) & (deuce_backhand_return_points['lastShotResult'] != 'Error'))])

# Count for Deuce Return Points - Error
count_deuce_forehand_error = len(deuce_forehand_return_points[(deuce_forehand_return_points['lastShotResult'] == 'Error') & (deuce_forehand_return_points['rallyCount'] == 2)])
count_deuce_backhand_error = len(deuce_backhand_return_points[(deuce_backhand_return_points['lastShotResult'] == 'Error') & (deuce_backhand_return_points['rallyCount'] == 2)])

# Display the counts
print("\nDeuce Forehand Return Points - Made:", count_deuce_forehand_made)
print("Deuce Forehand Return Points - Error:", count_deuce_forehand_error)

print("Deuce Backhand Return Points - Made:", count_deuce_backhand_made)
print("Deuce Backhand Return Points - Error:", count_deuce_backhand_error)

# Assuming return_points is your DataFrame
ad_return_points = return_points[(return_points['side'] == 'Ad') & (return_points['returnerName'] == first_player1Name) & (return_points['rallyCount'] >= 2)]

# Ad Return Points Separated by returnFhBh
ad_forehand_return_points = ad_return_points[ad_return_points['returnFhBh'] == 'Forehand']
ad_backhand_return_points = ad_return_points[ad_return_points['returnFhBh'] == 'Backhand']

# Count for Ad Return Points - Made
count_ad_forehand_made = len(ad_forehand_return_points[(ad_forehand_return_points['rallyCount'] > 2) | ((ad_forehand_return_points['rallyCount'] == 2) & (ad_forehand_return_points['lastShotResult'] != 'Error'))])
count_ad_backhand_made = len(ad_backhand_return_points[(ad_backhand_return_points['rallyCount'] > 2) | ((ad_backhand_return_points['rallyCount'] == 2) & (ad_backhand_return_points['lastShotResult'] != 'Error'))])

# Count for Ad Return Points - Error
count_ad_forehand_error = len(ad_forehand_return_points[(ad_forehand_return_points['lastShotResult'] == 'Error') & (ad_forehand_return_points['rallyCount'] == 2)])
count_ad_backhand_error = len(ad_backhand_return_points[(ad_backhand_return_points['lastShotResult'] == 'Error') & (ad_backhand_return_points['rallyCount'] == 2)])

# Display the counts
print("\nAd Forehand Return Points - Made:", count_ad_forehand_made)
print("Ad Forehand Return Points - Error:", count_ad_forehand_error)

print("Ad Backhand Return Points - Made:", count_ad_backhand_made)
print("Ad Backhand Return Points - Error:", count_ad_backhand_error)

print(f"\nAt Net Results for {first_player1Name}:\n")


# Total points where atNetPlayer1 = 1
total_at_net_player1 = len(point_df[point_df['atNetPlayer1'] == 1])

# Percentage of points where atNetPlayer1 = 1 out of total points
percentage_at_net_player1 = (total_at_net_player1 / len(point_df)) * 100 if len(point_df) > 0 else 0

# Display the total count and percentage of points where atNetPlayer1 = 1
print(f"Total Net Points for {first_player1Name}: {total_at_net_player1}")
print(f"Percentage of Net Points for {first_player1Name}: {percentage_at_net_player1:.2f}%")

# Points where atNetPlayer1 = 1 and pointWonBy = first_player1Name
at_net_player1_and_won_by_player1 = len(point_df[(point_df['atNetPlayer1'] == 1) & (point_df['pointWonBy'] == first_player1Name)])

# Percentage of points where atNetPlayer1 = 1 and pointWonBy = first_player1Name out of total points where atNetPlayer1 = 1
percentage_at_net_player1_and_won_by_player1 = (at_net_player1_and_won_by_player1 / total_at_net_player1) * 100 if total_at_net_player1 > 0 else 0

# Display the count and percentage of points where atNetPlayer1 = 1 and pointWonBy = first_player1Name
print(f"\nTotal Net Points won by {first_player1Name}: {at_net_player1_and_won_by_player1}")
print(f"Percentage of Net Points won by {first_player1Name}: {percentage_at_net_player1_and_won_by_player1:.2f}%")


In [ ]:
# Filter points where serverName is equal to first_player1Name
filtered_points = point_df[point_df['serverName'] == first_player1Name]

# Group the filtered points by player1ServePlacement and count the occurrences
serve_placement_counts = filtered_points.groupby('player1ServePlacement').size()


# Iterate over filtered_points
for index, point in filtered_points.iterrows():
    serve_placement = point['player1ServePlacement']
    
    # Check if serve placement is not in serve_placement_counts
    if serve_placement not in serve_placement_counts:
        print(point)


# Initialize dictionaries to store counts and percentages
point_won_counts = {}
point_won_percentages = {}
print(f"Total {len(filtered_points)}")

# Iterate over serve placements
for serve_placement, count in serve_placement_counts.items():
    # Filter points with the specific serve placement
    serve_placement_points = filtered_points[filtered_points['player1ServePlacement'] == serve_placement]
    
    # Count points won by first_player1Name
    point_won_count = serve_placement_points[serve_placement_points['pointWonBy'] == first_player1Name].shape[0]
    
    # Calculate percentage
    point_won_percentage = (point_won_count / count) * 100 if count > 0 else 0
    
    # Store counts and percentages
    point_won_counts[serve_placement] = point_won_count
    point_won_percentages[serve_placement] = point_won_percentage

# Print counts and percentages
for serve_placement, count in serve_placement_counts.items():
    print(f"Serve Placement: {serve_placement}")
    print(f"Total Serves: {count}")
    print(f"Serves Won by {first_player1Name}: {point_won_counts.get(serve_placement, 0)}")
    print(f"Percentage: {point_won_percentages.get(serve_placement, 0):.2f}%\n")

In [ ]:
print(f"\nError Data for {first_player1Name}:\n")
# Filter the DataFrame based on specified conditions
total_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) & 
                           (point_df['lastShotResult'] == 'Error')]

import numpy as np

# Filter rows without NaN values in relevant columns
forehand_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) & 
                           (point_df['lastShotResult'] == 'Error') &
                           (point_df['lastShotFhBh'] == 'Forehand') &
                           (~point_df['errorType'].isnull())]  # Ensure 'errorType' column doesn't have NaN

backhand_errors = point_df[(point_df['lastShotHitBy'] == first_player1Name) & 
                           (point_df['lastShotResult'] == 'Error') &
                           (point_df['lastShotFhBh'] == 'Backhand') &
                           (~point_df['errorType'].isnull())]  # Ensure 'errorType' column doesn't have NaN

# Count the occurrences of 'Forehand' and 'Backhand' separately
forehand_counts = forehand_errors.shape[0]  # Count rows
backhand_counts = backhand_errors.shape[0]  # Count rows

# Print the total error counts for verification
total_error_counts = forehand_counts + backhand_counts

# Get value counts of 'errorType' for Forehand errors
forehand_error_types = forehand_errors['errorType'].value_counts(dropna=False)  # Include NaN values in count

# Get value counts of 'errorType' for Backhand errors
backhand_error_types = backhand_errors['errorType'].value_counts(dropna=False)  # Include NaN values in count

# Print the counts and error types
print("Count of Total errors:", total_error_counts)
print("Count of Forehand errors:", forehand_counts)
print("Forehand Error %:", forehand_counts/total_error_counts)
print("Count of Backhand errors:", backhand_counts)
print("Backhand Error %:", backhand_counts/total_error_counts)
print("\nForehand errors:\n", forehand_error_types)


# # Group by both 'lastShotDirection' and 'errorType', and then count occurrences
forehand_error_counts = forehand_errors.groupby(['player1LastShotPlacement', 'errorType']).size().unstack(fill_value=0)  # Fill NaN with 0

print("\nValue counts of 'errorType' for Forehand errors with different directions:\n", forehand_error_counts)


print("\nBackhand errors:\n", backhand_error_types)



# Group by both 'lastShotDirection' and 'errorType', and then count occurrences
backhand_error_counts = backhand_errors.groupby(['player1LastShotPlacement', 'errorType']).size().unstack(fill_value=0)  # Fill NaN with 0

print("\nValue counts of 'errorType' for Backhand errors with different directions:\n", backhand_error_counts)